In [1]:
#-----------------Geometric Method--------------------------
#X,Y,phi = Camera based
#Z = Sonar Based
#import the values found from the object detector (x,y,z,Phi)

import numpy as np
import math

data1 = open("Theta1.txt","w+")
data2 = open("Theta2.txt","w+")
data3 = open("Theta3.txt","w+")
data4 = open("Theta4.txt","w+")

# the time function which defines the path motion will be dependent on how many iterations, which will define how smoothly the robot moves.
time = np.arange(0,5,1)
x = [0.300, 0.15, 0.0, -0.15, -0.30]
y = [0, 0.1, 0.2, 0.1, 0]
z = [0.000, 0.10, 0.2, 0.100, 0.000]             


#For ODA Three inputs
# 1. End effector of object location
# 2. End effector height based on bucket or categorization location
# 3. End effector end locaton based on categorization
# a. Release of gripper of location
# b. Return location

#Link Lengths in cm.
l1 = 13.0 
l2 = 13.0
l3 = 19.0

for k in time:
    #Orientation of end effector as a function of time
    xe = x[k]*10
    ye = y[k]*10
    ze = z[k]*10
    
    phie = -45
    
    xp = xe - l3*np.cos(np.radians(phie))
    yp = ye - l3*np.sin(np.radians(phie))
    
    #if xe==0 or ze==0:
    #    Theta1 = 0
    #else:
    #    Theta1 = np.arctan2(ze, xe)
        
    Theta1 = np.arctan2(ze,xe)
    
    print(-(xp**2 + yp**2 + l1**2 - l2**2)/(2*l1*(xp**2 + yp**2)**0.5))
    
    Theta2 = np.arctan2((-yp)/(xp**2 + yp**2)**0.5, xp/(xp**2 + yp**2)**0.5) + np.arccos(-(xp**2 + yp**2 + l1**2 - l2**2)/(2*l1*(xp**2 + yp**2)**0.5))
    
    Theta3 = np.arctan2((yp - l1*np.sin((Theta2)))/(l2),(xp - l1*np.cos((Theta2)))/(l2)) - (Theta2)
    
    Theta4 = phie - Theta2 - Theta3
    
    Theta1 = Theta1 * 180/np.pi
    
    # To account for y-axis measurement to the x-axis in ADAMS measurement. This is due to the coordinate frames present in dereiviation
    Theta2 = Theta2 * 180/np.pi - 90
    
    #This sets the boundary conditions so it does not touch the floor.
    #if Theta2 > 200:
    #    Theta2 = 200
    #elif Theta2 < -20:
    #    Theta2 = -20
        
        
    #Data Check
    print(" End Effector Coordinates x =", xe ," y = ",ye, " phi = ", phie)
    print("Theta 1 ", np.round(Theta1) , " at time ", k," seconds")
    print("Theta 2 ", np.round(Theta2) , " at time ", k," seconds")
    print("Theta 3 ", np.round(Theta3), " at time ", k," seconds")
    print("Theta 4 ", np.round(Theta4), " at time ", k," seconds")
    
    #Preventing nan situations and allowing the spline to fit the function in order to assure continuity in the path function
    if math.isnan(Theta2) == False:
        data1.write(str(k) + " " + str(Theta1) + '\n')
        data2.write(str(k) + " " + str(Theta2) + '\n')
        data3.write(str(k) + " " + str(Theta3) + '\n')
        data4.write(str(k) + " " + str(Theta4) + '\n')
        

data1.close()
data2.close()
data3.close()
data4.close()




-0.654286985945324
 End Effector Coordinates x = 3.0  y =  0  phi =  -45
Theta 1  0.0  at time  0  seconds
Theta 2  -87.0  at time  0  seconds
Theta 3  3.0  at time  0  seconds
Theta 4  -48.0  at time  0  seconds
-0.7203867419138349
 End Effector Coordinates x = 1.5  y =  1.0  phi =  -45
Theta 1  34.0  at time  1  seconds
Theta 2  -83.0  at time  1  seconds
Theta 3  3.0  at time  1  seconds
Theta 4  -48.0  at time  1  seconds
-0.787043861858047
 End Effector Coordinates x = 0.0  y =  2.0  phi =  -45
Theta 1  90.0  at time  2  seconds
Theta 2  -79.0  at time  2  seconds
Theta 3  2.0  at time  2  seconds
Theta 4  -48.0  at time  2  seconds
-0.7988760079351003
 End Effector Coordinates x = -1.5  y =  1.0  phi =  -45
Theta 1  146.0  at time  3  seconds
Theta 2  -83.0  at time  3  seconds
Theta 3  3.0  at time  3  seconds
Theta 4  -48.0  at time  3  seconds
-0.8164454030953946
 End Effector Coordinates x = -3.0  y =  0  phi =  -45
Theta 1  180.0  at time  4  seconds
Theta 2  -86.0  at time 

In [41]:
import time
import numpy as np
import math
import serial

serialPort = serial.Serial(port = "COM4",baudrate=115200, bytesize=8,stopbits=serial.STOPBITS_ONE)
print(serialPort)
#help(serialPort)

#k = -1
#while end==False:
    #Detection input will relay
    #k = k + 1
    #xinput, yinput, zinput, phiinput = open("Sensor Parameters.txt","w+")
    #Ordinalclass = open("Classification Parameters")
    #0: King
    #1: Queen
    #2: Bishop
    #3: Knight
    #4: Rook
    #5: Pawn
    #Ordinalclass = {0: xe = ,ye = ,ze = ,phie = 
    #                1: xe = ,ye = ,ze = ,phie = 
    #                2: xe = ,ye = ,ze = ,phie = 
    #                3: xe = ,ye = ,ze = ,phie = 
    #                4: xe = ,ye = ,ze = ,phie = 
    #                5: xe = ,ye = ,ze = ,phie = 
    #                 }
        
def InverseKinematics(x,y,z,phi): # input must be in dm
    #Link Lengths
    l1 = 13.0 
    l2 = 13.0
    l3 = 19.0

    xp = x - l3*np.cos(np.radians(phi))
    yp = y - l3*np.sin(np.radians(phi))

    Theta1 = np.arctan2(z,x)

    Theta2 = np.arctan2((-yp)/(xp**2 + yp**2)**0.5, xp/(xp**2 + yp**2)**0.5) + np.arccos(-(xp**2 + yp**2 + l1**2 - l2**2)/(2*l1*(xp**2 + yp**2)**0.5))

    Theta3 = np.arctan2((yp - l1*np.sin((Theta2)))/(l2),(xp - l1*np.cos((Theta2)))/(l2)) - (Theta2)

    Theta4 = phi - Theta2 - Theta3

    # To account for y-axis measurement to the x-axis in ADAMS measurement. This is due to the coordinate frames present in dereiviation
    Theta2 = Theta2 * 180/np.pi - 90
    Theta1 = Theta1 * 180/np.pi
    
    #Data Check
    print("\n End Effector Coordinates x =", x ," y = ",y," z = ",z, " phi = ", phi)
    print("Theta 1 ", np.round(Theta1) , " at time ")
    print("Theta 2 ", np.round(Theta2) , " at time ")
    print("Theta 3 ", np.round(Theta3), " at time ")
    print("Theta 4 ", np.round(Theta4), " at time")

    #Preventing nan situations and allowing the spline to fit the function in order to assure continuity in the path function
    if math.isnan(Theta2) == False:
        Angles = np.array([abs(Theta1), abs(Theta2), abs(Theta3), abs(Theta4)])
        return Angles

# the inital end effector coordnates and the final end effecor cooridnates with a given phi. t is the time interval for the frequency of sending pules of data
def Spline(xi,yi,zi,xe,ye,ze,phi,n,t):# n is the number of intervals desired between the input and output location
    intervalx = (xi - xe)/n
    intervaly = (yi - ye)/n
    intervalz = (zi - ze)/n 
    route = True
    xs = xi
    ys = yi
    zs = zi
    count = 0
    while route==True:
        count = count + 1
        print("Iteration: ", count)
        if count == 1:
            RB = 816
            RB_array = np.array([RB], dtype = np.single)
            RB_binary = RB_array.tobytes()
            voyager = np.array([160,5,RB_binary[0],0,0,0,176,177], dtype = np.uint8)
        
        xs = xs + intervalx
        ys = ys + intervaly
        zs = zs + intervalz
        
        JointAngles = InverseKinematics(xs,ys,zs,phi)
        #for t in np.arange(len(JointAngles)):
        #    if JointAngles[t] < 816:
        #        JointAngles[t] = JointAngles[t] + 816
                
        Time = JointAngles*0.01031 + 0.544
        RB = Time*24e3/16
        RB = (RB**2)**0.5
        
        RB_array = np.array(RB, dtype = np.single)
        RB_binary = RB_array.tobytes() 
        #serialPort.reset_output_buffer()
        print("RB actual value", RB)
        print("RB_Array: ",RB_array[0], RB[1],RB[2],RB[3])
        print('len = ',len(RB_binary),RB_binary[0], RB_binary[1], RB_binary[2], RB_binary[3])
        
        for i in np.arange(1,5,1):
            print("RB value for motor ",i," with an RB value of: ", RB[i-1], "sending at a frequency of: ", 1/t, "Hz")
            #voyager = np.array([160,2,300,0,0,0,176,177], dtype = np.uint8)
            print("RB value sending",RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3])
            voyager = np.array([160,i,RB_binary[(i-1)*4],RB_binary[(i-1)*4+1],RB_binary[(i-1)*4+2],RB_binary[(i-1)*4+3],176,177], dtype = np.uint8)
            print(len(voyager))
            serialPort.write(voyager.data)     
            time.sleep(t)
            
        if count == n:
            RB = 2208
            RB_array = np.array([RB], dtype = np.single)
            RB_binary = RB_array.tobytes()
            #voyager = np.array([160,5,RB_binary[0],0,0,0,176,177], dtype = np.uint8)
            route = False
        
        
Spline(3.0,0,0,5.0,2.0,0,-45,5,5)
serialPort.close()

Serial<id=0x252e7c1b198, open=True>(port='COM4', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=None, xonxoff=False, rtscts=False, dsrdtr=False)
Iteration:  1

 End Effector Coordinates x = 2.6  y =  -0.4  z =  0.0  phi =  -45
Theta 1  0.0  at time 
Theta 2  -89.0  at time 
Theta 3  3.0  at time 
Theta 4  -48.0  at time
RB actual value [ 816.         2193.10865667  856.69594531 1552.87823028]
RB_Array:  816.0 2193.108656669985 856.6959453069336 1552.8782302842094
len =  16 0 0 76 68
RB value for motor  1  with an RB value of:  816.0000000000001 sending at a frequency of:  0.2 Hz
RB value sending 0 0 76 68
8
RB value for motor  2  with an RB value of:  2193.108656669985 sending at a frequency of:  0.2 Hz
RB value sending 189 17 9 69
8
RB value for motor  3  with an RB value of:  856.6959453069336 sending at a frequency of:  0.2 Hz
RB value sending 138 44 86 68
8
RB value for motor  4  with an RB value of:  1552.8782302842094 sending at a frequency of:  0.2 Hz
RB value send

SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

In [35]:
Theta = InverseKinematics(0.30,0,0,-45)
print(Theta)
time = Theta*0.01031 + 0.544
RB = time*24e3/16
print(RB)
RB_array = np.array([RB], dtype = np.single)
RB_binary = RB_array.tobytes() 

for j in np.arange(1,5,1):
    voyager = np.array([160,j,RB_binary[j],0,0,0,176,177], dtype = np.uint8)
    serialPort.write(voyager.data)
serialPort.close()


 End Effector Coordinates x = 0.3  y =  0  z =  0  phi =  -45
Theta 1  0.0  at time 
Theta 2  -88.0  at time 
Theta 3  3.0  at time 
Theta 4  -48.0  at time
[  0.         -88.07887362   2.64639623 -47.67992621]
[ 816.         -546.13978058  856.92651765   78.6299412 ]


SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))

In [ ]:
import sys
import utils

In [ ]:
import numpy as np
import time
for k in np.arange(1,10,1):
    print(k)
    t = 1/k
    freq = 1/t
    print(freq,"Hz")
    
    time.sleep(t)
    


    

In [37]:
x = -2
print(abs(x))

2
